# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [2]:
ib.isConnected()

True

### Streaming tick data

Create some Forex contracts:

In [3]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [4]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [6]:
# ticker = ib.tickebr(eurusd)
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2018, 8, 16, 17, 49, 26, 957845, tzinfo=datetime.timezone.utc), bid=1.1352, bidSize=4000000, ask=1.13521, askSize=1000000, prevBid=1.13521, prevBidSize=1000000, prevAsk=1.13523, prevAskSize=4000000, high=1.14095, low=1.13355, close=1.1345, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [7]:
ticker.marketPrice()

1.135195

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [8]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]
contract2Row = {c: i for (i, c) in enumerate(contracts)}

def onPendingTickers(tickers):
    for t in tickers:
        row = contract2Row[t.contract]
        df.iloc[row, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,1000000,1.1352,1.13523,15000000,1.14095,1.13355,1.1345
1,USDJPY,3000000,110.911,110.913,3000000,111.12,110.455,110.74
2,GBPUSD,1000000,1.26964,1.26969,1000000,1.2754,1.26855,1.2698
3,USDCHF,4000000,0.99705,0.9971,2000000,0.99735,0.9916,0.99365
4,USDCAD,1000000,1.31692,1.31696,1000000,1.3171,1.3114,1.314
5,AUDUSD,1000000,0.72539,0.72542,2000000,0.7287,0.72155,0.72395


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [9]:
for contract in contracts:
    ib.cancelMktData(contract)

### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [8]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2018, 6, 6, 13, 54, 34, tzinfo=datetime.timezone.utc), mask=0, priceBid=1.17906, priceAsk=1.17908, sizeBid=3500000, sizeAsk=2000000)

In [10]:
ib.disconnect()

In [11]:
df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]
contract2Row = {c: i for (i, c) in enumerate(contracts)}

def onPendingTickers(tickers):
    for t in tickers:
        row = contract2Row[t.contract]
        df.iloc[row, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

False